# Модуль 3. Мультимодальные и мультизадачные модели. Часть 1

Это вторая часть домашней работы №3 "Реализация Visual Question Answering / Document Question Answering"

## Часть 2. Использование модели

**Цель:** отработать навыки адаптации готовых моделей для решения прикладной задачи на русском языке, а также создание небольших демо для задач.

**В каком виде прислать результат:**

заполненный jupyter-notebook и видеозаписи работы с демо

### [2 балла] Добавить модель переводчика

У вас уже есть готовая модель, которая может по картинке отвечать на текстовые запросы к картинке. Ваша цель --- обобщить эту модель на русский язык, добавив модель переводчик, которая будет переводить запрос на русском языке в запрос на английском языке и передавать его модели. За основу вы можете взять языковую модель (например, https://huggingface.co/Helsinki-NLP/opus-mt-ru-en). Альтернативой может стать реализация функции, делающий api вызов, к приложению переводчика (например, https://libretranslate.com/).

---

**Ожидаемый результат**

В качестве результата в этой секции вам нужно предоставить функции, которые делают перевод с русского на английского и делает инференс модели DocVQA и выводит ответ на русском языке. (В качестве примеров вопросов, можете использовать данные из датасета)


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from huggingface_hub import Repository

# Путь к локальной папке, где вы храните файлы для загрузки
local_folder = r"C:/Users/NeKonn/MFTI/CMMO/HW_3/layoutlmv2-base-uncased_finetuned_docvqa"

# Имя вашего репозитория на Hugging Face Hub
repo_name = "layoutlmv2-base-uncased_finetuned_docvqa"

# Клонирование репозитория
# Убедитесь, что используете своё имя пользователя вместо NeKonnn, если оно другое
repo = Repository(local_folder, clone_from=f"NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa")

# Пуш файлов в репозиторий
repo.git_add()
repo.git_commit("Initial commit")
repo.git_push()

OSError: Tried to clone https://huggingface.co/NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa in an unrelated git repository.
If you believe this is an error, please add a remote with the following URL: https://huggingface.co/NeKonnn/layoutlmv2-base-uncased_finetuned_docvqa.
Local path has its origin defined as: https://huggingface.co/Mikhail1313/layoutlmv2-base-uncased_finetuned_docvqa


In [2]:
! pip install -q transformers datasets

In [3]:
! pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00


In [4]:
!pip install -q git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!pip install -q opencv-python

  Preparing metadata (setup.py) ... done


In [8]:
!pip install -q git+https://github.com/facebookresearch/detectron2.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
! pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into accou

In [29]:
!pip install -q pillow

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import MarianMTModel, MarianTokenizer

# Загрузка модели и токенизатора для перевода с русского на английский
model_name = "Helsinki-NLP/opus-mt-ru-en"
model_translate_ru2en = MarianMTModel.from_pretrained(model_name)
tokenizer_translate_ru2en = MarianTokenizer.from_pretrained(model_name)

# Функция для перевода с русского на английский
def translate_ru2en(text):
    inputs = tokenizer_translate_ru2en(text, return_tensors="pt")
    outputs = model_translate_ru2en.generate(**inputs)
    translated_text = tokenizer_translate_ru2en.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Проверка
text = "Привет, как дела?"
translated_text = translate_ru2en(text)
print("Original:", text)
print("Translated:", translated_text)

Original: Привет, как дела?
Translated: Hey, how's it going?


In [4]:
# Загрузка модели и токенизатора для перевода с английского на русский
model_en2ru_name = "Helsinki-NLP/opus-mt-en-ru"
model_en2ru = MarianMTModel.from_pretrained(model_en2ru_name)
tokenizer_en2ru = MarianTokenizer.from_pretrained(model_en2ru_name)

# Функция для перевода с английского на русский
def translate_en2ru(text):
    inputs = tokenizer_en2ru(text, return_tensors="pt")
    outputs = model_en2ru.generate(**inputs)
    translated_text = tokenizer_en2ru.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Проверка функции
text = "Hey, how's it going?"
translated_text = translate_en2ru(text)
print("Original:", text)
print("Translated:", translated_text)

Original: Hey, how's it going?
Translated: Привет, как дела?


In [5]:
from transformers import LayoutLMv2Processor, LayoutLMv2ForQuestionAnswering, LayoutLMv2Tokenizer
import torch

# Загрузка модели, процессора и токенезатора
model_docvqa = LayoutLMv2ForQuestionAnswering.from_pretrained("Mikhail1313/layoutlmv2-base-uncased_finetuned_docvqa")
processor_docvqa = LayoutLMv2Processor.from_pretrained("Mikhail1313/layoutlmv2-base-uncased_finetuned_docvqa")
tokenizer_docvqa = LayoutLMv2Tokenizer.from_pretrained("Mikhail1313/layoutlmv2-base-uncased_finetuned_docvqa")

In [6]:
from PIL import Image

def ru_inference(image_path, question_ru):

    image = Image.open(image_path).convert("RGB") # Загрузка изображения

    question_en = translate_ru2en(question_ru) # Перевод вопроса с русского на английский

    inputs = processor_docvqa(image, question_en, return_tensors="pt") # Препроцессинг изображения и вопроса

    outputs = model_docvqa(**inputs) # Инференс модели

    # Извлечение ответа
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Игнорирование служебных токенов при выборе начального и конечного индексов
    ignore_index = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
    answer_start_scores[:, ignore_index] = -float("Inf")
    answer_end_scores[:, ignore_index] = -float("Inf")
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]) # Извлечение токенов ответа

    answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)] # Удаление служебных токенов из ответа

    answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens) # Преобразование токенов в строку

    answer_ru = translate_en2ru(answer) # Перевод ответа с английского на русский
    return answer_ru

In [7]:
answer = ru_inference("/content/What.jpg", "Что такое искусственный интеллект?")
print("Ответ:", answer)

Ответ: Искусственный интеллект - это исследование систем, которые действуют таким образом, что для любого наблюдателя представляется разумным. На самом деле, методы искусственного интеллекта часто используются для решения относительно простых проблем или сложных проблем, которые являются внутренними для более сложных систем. Это может привести нас к другому определению искусственного интеллекта, а именно: искусственный интеллект предполагает использование методов, основанных на разумном поведении людей и других животных, для решения сложных проблем.


### [2 балла] Сделать демо на gradio

Модель готова! Теперь было бы круто, если модель можно было захостить и оттестировать на практике. В этом задании вам нужно будет реализовать демо на gradio, которое будет принимать изображение и вопрос, а далее выдавать ответ. Пример демо, аналогично которому вам нужно реализовать модель --- https://huggingface.co/spaces/nielsr/comparing-VQA-models.


**Подсказка:**

В вкладке `Files` на демо вы можете посмотреть реализацию, там нужно заменить инференс, используемой модели, на инференс нашей модели с переводом


**Ожидаемый результат**

В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал. Видео прикрепляйте отдельным файлом.

In [24]:
# переопределим немного функцию

import torchvision.transforms as transforms

def ru_inference(image, question_ru):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0)  # Преобразование изображения в тензор PyTorch

    question_en = translate_ru2en(question_ru)  # Перевод вопроса с русского на английский

    inputs = processor_docvqa(image, question_en, return_tensors="pt")  # Препроцессинг изображения и вопроса

    outputs = model_docvqa(**inputs)  # Инференс модели

    # Извлечение ответа
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Игнорирование служебных токенов при выборе начального и конечного индексов
    ignore_index = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
    answer_start_scores[:, ignore_index] = -float("Inf")
    answer_end_scores[:, ignore_index] = -float("Inf")
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])  # Извлечение токенов ответа

    answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)]  # Удаление служебных токенов из ответа

    answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens)  # Преобразование токенов в строку

    answer_ru = translate_en2ru(answer)  # Перевод ответа с английского на русский
    return answer_ru

In [25]:
import gradio as gr
import torch

# Определение интерфейса Gradio
iface = gr.Interface(
    fn=ru_inference,  # функция, которую нужно вызвать
    inputs=[gr.Image(type="pil"), gr.Textbox(lines=2, placeholder="Введите вопрос на русском языке")],  # входные данные: изображение и текст
    outputs=gr.Textbox(),  # выходные данные: текст
)

# Запуск Gradio интерфейса
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://bf953a2e89605732e1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://bf953a2e89605732e1.gradio.live


### [4 балла] Ответы на вопросы голосом

Демо готово! Но кто хочет писать вопросы текстом?
Здесь вам предстоить улучшить ваше демо, чтобы оно могло принимать вопросы голосом. За основу вам предлагается рассмотреть демо https://www.gradio.app/guides/real-time-speech-recognition и добавить соответствуещее окошко в ваше демо. Также вы можете добавить text-to-speech модель, чтобы оно озвучило текстовый ответ (дополнительный балл к оценке)

---

**Ожидаемый результат**

В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал.

In [26]:
!pip install -q SpeechRecognition pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 23.6 MB/s eta 0:00:00


In [31]:
import speech_recognition as sr

def recognize_speech(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
    text = r.recognize_google(audio, language='ru-RU')
    return text

In [32]:
def ru_inference(image, question_ru=None, audio=None):
    if audio is not None:
        question_ru = recognize_speech(audio)

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    image = transform(image).unsqueeze(0)  # Преобразование изображения в тензор PyTorch

    question_en = translate_ru2en(question_ru)  # Перевод вопроса с русского на английский

    inputs = processor_docvqa(image, question_en, return_tensors="pt")  # Препроцессинг изображения и вопроса

    outputs = model_docvqa(**inputs)  # Инференс модели

    # Извлечение ответа
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Игнорирование служебных токенов при выборе начального и конечного индексов
    ignore_index = [tokenizer_docvqa.cls_token_id, tokenizer_docvqa.sep_token_id, tokenizer_docvqa.pad_token_id]
    answer_start_scores[:, ignore_index] = -float("Inf")
    answer_end_scores[:, ignore_index] = -float("Inf")
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer_tokens = tokenizer_docvqa.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])  # Извлечение токенов ответа

    answer_tokens = [token for token in answer_tokens if token not in (tokenizer_docvqa.cls_token, tokenizer_docvqa.sep_token, tokenizer_docvqa.pad_token)]  # Удаление служебных токенов из ответа

    answer = tokenizer_docvqa.convert_tokens_to_string(answer_tokens)  # Преобразование токенов в строку

    answer_ru = translate_en2ru(answer)  # Перевод ответа с английского на русский
    return answer_ru

In [34]:
iface = gr.Interface(
    fn=ru_inference,
    inputs=[
        gr.Image(type="pil"),
        gr.Textbox(lines=2, placeholder="Введите вопрос на русском языке (или оставьте пустым, если хотите использовать голос)"),
        gr.Audio(source="microphone", type="filepath", optional=True)
    ],
    outputs=gr.Textbox(),
)

iface.launch(debug=True)

<ipython-input-34-0008f800c3c8>:6: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Audio(source="microphone", type="filepath", optional=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://58401b3631a51fba21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://58401b3631a51fba21.gradio.live
